<a href="https://colab.research.google.com/github/IoriGunji-Vantiq/TempPublic/blob/master/MqttPublisherSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MQTT Publisher Sample
MQTTブローカーへメッセージの送信を行うサンプルスクリプトになります。  
設定したメッセージをランダムで送信します。  

## 設定方法
利用する際は事前にいくつか設定が必要になります。  
また、事前に Google へのログインが必要になります。  

### MQTTブローカーの設定
`# MQTTブローカー設定` に必要な情報を入力します。  
- `broker`：ホスト名を入力します。
- `port`：MQTTのポート番号を入力します。
- `topic`：トピック名を入力します。
- `client_id`：クライアントIDは、特段の事情がない限り変更不要です。
- `username`：ユーザー名を設定しない場合は、設定不要です。
- `password`：パスワードを設定しない場合は、設定不要です。

### 送信データの設定
`# 送信データ設定` に必要な情報を入力します。
- `interval`：メッセージの送信間隔を秒単位で指定します。（送信頻度を変更したい場合以外は変更不要）
- `message`：送信したいメッセージをオブジェクトの配列で入力します。

## メッセージの送信方法
1. 初回実行時または実行時にエラーが出る場合は、モジュールのインストールを行います。  
   1. `# モジュールのインストール` の左側の実行ボタンをクリックします。  
1. モジュールのインポートを行います。  
   1. `# モジュールインポート` の左側の実行ボタンをクリックします。  
1. 設定情報を反映させます。  
   1. `# MQTTブローカー設定` の左側の実行ボタンをクリックします。
   1. `# 送信データ設定` の左側の実行ボタンをクリックします。  
1. メッセージの送信を行います。  
   1. `# MQTT Publisher 本体` の左側の実行ボタンをクリックします。

## メッセージ送信の停止方法
1. `# MQTT Publisher 本体` の左側の停止ボタンをクリックします。

In [ ]:
# モジュールのインストール
!pip install paho-mqtt

In [ ]:
# モジュールインポート
import time
import json
import random
from datetime import datetime as dt
from paho.mqtt import client as mqtt_client

In [ ]:
# MQTTブローカー設定
mqtt_config = {
    'broker': 'broker.hivemq.com'
    , 'port': 1883
    , 'topic': '/topic_name'
    , 'client_id': f'python-mqtt-{random.randint(0, 100)}'
    , 'username': ''
    , 'password': ''
}

In [ ]:
# 送信データ設定
publish_config = {
    'interval': 2
    , 'message': [
        {
            'name': 'シナモン'
            , 'birthday': '3月6日'
        }
        , {
            'name': 'カプチーノ'
            , 'birthday': '6月27日'
        }
        , {
            'name': 'モカ'
            , 'birthday': '2月20日'
        }
        , {
            'name': 'シフォン'
            , 'birthday': '1月14日'
        }
        , {
            'name': 'エスプレッソ'
            , 'birthday': '12月4日'
        }
        , {
            'name': 'みるく'
            , 'birthday': '2月4日'
        }
    ]
}

In [ ]:
# MQTT Publisher 本体
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!\n")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(mqtt_config['client_id'])
    client.username_pw_set(mqtt_config['username'], mqtt_config['password'])
    client.on_connect = on_connect
    client.connect(mqtt_config['broker'], mqtt_config['port'])
    return client


def publish(client):
    msg_length = len(publish_config['message'])
    while True:
        time.sleep(publish_config['interval'])
        msg = publish_config['message'][random.randint(0, msg_length - 1)]
        msg['time'] = dt.now().strftime('%Y-%m-%d %H:%M:%S')
        result = client.publish(mqtt_config['topic'], json.dumps(msg, ensure_ascii=False, indent=4))
        status = result[0]
        if status == 0:
            print(f"Topic: {mqtt_config['topic']}")
            print(f"{msg}")
            print(f"Published Time: {msg['time']}")
            print()
        else:
            print(f"Failed to send message to topic {mqtt_config['topic']}")


def run():
    client = connect_mqtt()
    client.loop_start()
    publish(client)


if __name__ == '__main__':
    run()
